In [7]:
import contentful
import pandas as pd

In [8]:
SPACE_ID = 'vhjns0a2q244'
ACCESS_TOKEN = '189c6b77fa686e8bfc15ae9bc63e0149b269a1715974f0ba5345adc5dadb65b8'

client = contentful.Client(SPACE_ID, ACCESS_TOKEN)

entries = client.entries({'content_type': 'articles', 'limit': 300})
entries

<Array size='216' total='216' limit='300' skip='0'>

In [9]:
def get_values(body, result=None):
    """
    Извлечение текста из рекурсивной структуры данных
    """
    if result is None:
        result = [] 
    if 'content' in body:
        for item in body['content']:
            get_values(item, result)
    if 'value' in body:
        result.append(body['value'])
    return result

In [10]:
def clean_text(body):
    text = ' '.join(get_values(body))
    text = text.replace('\n\n', ' ')
    text = text.replace('\xa0', ' ')
    return text

Классифицированной считается та статья, которая размечина тэгами (метками) 2-ого уровня или выше.

In [11]:
def get_labels(industries):
    """
    Извлечение меток
    """
    tags = []
    industries = industries.split(';')
    for industry in industries:
        categories = industry.split('/')
        if len(categories) > 1:
            tags.append(categories[1])
    return tags

In [12]:
data = []

for item in entries:
    classes = []
    if hasattr(item, 'industries'):
        classes = get_labels(item.industries)
    data.append([item.id, clean_text(item.body), '/'.join(classes)])

In [13]:
df = pd.DataFrame(data, columns=['id', 'text', 'classes'])
df.head()

,id,text,classes
0,6nSIEp3ZfO4okkUaGmUksW,Строительная отрасль начала падение задолго до...,
1,3VGhiB5MEwGqG4MQiKMw6U,Бизнес часто использует номинальных руководите...,
2,28C2BJvmEsQKiS4AEO2eIE,За период с сентября 2017 года по октябрь 2018...,FMCG
3,65s7gPkLFmgkeIQQWCCcWW,"Также аналитики Knight Frank прогнозируют, что...",Недвижимость/Хранение
4,2ldCvrzcf6wcwOMwiI2A4U,Несмотря на то что основные экономические пока...,FMCG


In [14]:
df.to_csv('dataset.csv', index=False)

In [15]:
df = pd.read_csv('dataset.csv')
df.head()

,id,text,classes
0,6nSIEp3ZfO4okkUaGmUksW,Строительная отрасль начала падение задолго до...,NaN
1,3VGhiB5MEwGqG4MQiKMw6U,Бизнес часто использует номинальных руководите...,NaN
2,28C2BJvmEsQKiS4AEO2eIE,За период с сентября 2017 года по октябрь 2018...,FMCG
3,65s7gPkLFmgkeIQQWCCcWW,"Также аналитики Knight Frank прогнозируют, что...",Недвижимость/Хранение
4,2ldCvrzcf6wcwOMwiI2A4U,Несмотря на то что основные экономические пока...,FMCG
